In [1]:
import math
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Tree definition
def create_tree_polygon(x, y, angle_deg):
    """Create tree polygon at given position and angle."""
    # Tree vertices (relative to center at origin)
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    vertices = [
        (0.0, tip_y),
        (top_w/2, tier_1_y),
        (top_w/4, tier_1_y),
        (mid_w/2, tier_2_y),
        (mid_w/4, tier_2_y),
        (base_w/2, base_y),
        (trunk_w/2, base_y),
        (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, base_y),
        (-base_w/2, base_y),
        (-mid_w/4, tier_2_y),
        (-mid_w/2, tier_2_y),
        (-top_w/4, tier_1_y),
        (-top_w/2, tier_1_y),
    ]
    
    # Create polygon
    poly = Polygon(vertices)
    
    # Rotate
    poly = affinity.rotate(poly, angle_deg, origin=(0, 0))
    
    # Translate
    poly = affinity.translate(poly, x, y)
    
    return poly

# Visualization
def plot_packing(trees, title="Packing"):
    fig, ax = plt.subplots(figsize=(10, 10))
    for tree in trees:
        x, y = tree.exterior.xy
        ax.plot(x, y)
        ax.fill(x, y, alpha=0.5)
    ax.set_aspect('equal')
    plt.title(title)
    plt.show()

print("Setup complete")

Setup complete
